In [ ]:
!nvidia-smi

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import pandas as pd
import numpy as np
import re
import librosa
import seaborn as sns
import os
import json
import IPython.display as ipd
import soundfile as sf
import torch
import h5py
import onnxruntime as ort
import openvino as ov
import torch.quantization.quantize_fx as quantize_fx

from glob import glob
from tqdm import tqdm
from pprint import pprint
from matplotlib import pyplot as plt
from itertools import chain
from os.path import join as pjoin
from torchaudio.transforms import AmplitudeToDB, MelSpectrogram
from copy import deepcopy
from sklearn.metrics import f1_score 

# from code_base.utils import parallel_librosa_load, groupby_np_array, stack_and_max_by_samples, macro_f1_similarity, N_CLASSES_2021_2022, N_CLASSES_2021, comp_metric, N_CLASSES_XC_LIGIT_SHORTEN, N_CLASSES_XC_LIGIT_EVEN_SHORTEN
# from code_base.utils.constants import SAMPLE_RATE
from code_base.utils.onnx_utils import ONNXEnsemble, convert_to_onnx
from code_base.models import WaveCNNClasifier, WaveCNNAttenClasifier, WaveTDNNClasifier
from code_base.datasets import WaveDataset, WaveAllFileDataset
from code_base.utils.swa import avarage_weights, delete_prefix_from_chkp
from code_base.inefernce import BirdsInference
from code_base.utils import load_json, compose_submission_dataframe, groupby_np_array, stack_and_max_by_samples, write_json
from code_base.utils.metrics import score_numpy
from code_base.utils.main_utils import get_device
%matplotlib inline


/home/vsydorskyi/src/bird_clef_2025/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib.util>:262: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.


`speechbrain` was not imported
`LEAF` was not imported


/home/vsydorskyi/src/bird_clef_2025/.venv/lib/python3.11/site-packages/nnAudio/Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(


# Clean logdirs

In [ ]:
# def filt_criteria(input):
#     num_folds = sum("fold_" in el for el in glob(pjoin(input, "*")))
#     if num_folds > 0 and num_folds < 5:
#         print(input, num_folds)
#         return False
#     else:
#         return True
    
# all_logdirs = glob("../logdirs/*")
# logdirs_to_remove = [el for el in all_logdirs if not filt_criteria(el)]

In [ ]:
# for el in logdirs_to_remove[:-1]:
#     !rm {el} -rf

# Config

In [ ]:
!ls -lt ../logdirs/ | head

In [ ]:
# print("Possible exps:\n\n{}".format("\n".join(os.listdir("../logdirs/"))))

In [ ]:
# bird2id_source = load_json("/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/birdclef_2025/bird2int_2025_and_prev_comps.json")
# bird2id_target = load_json("/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/birdclef_2025/bird2int_2025.json")

# id2bird_source = {v:k for k,v in bird2id_source.items()}
# id2bird_target = {v:k for k,v in bird2id_target.items()}

# REARRANGE_INDICES = np.array([
#     bird2id_source[id2bird_target[i]] for i in range(len(id2bird_target))
# ]).astype(int)

# def prune_checkpoint_rule(inp_chkp):
#     inp_chkp["head.attention.weight"] = inp_chkp["head.attention.weight"][REARRANGE_INDICES]
#     inp_chkp["head.attention.bias"] = inp_chkp["head.attention.bias"][REARRANGE_INDICES]
    
#     inp_chkp["head.fix_scale.weight"] = inp_chkp["head.fix_scale.weight"][REARRANGE_INDICES]
#     inp_chkp["head.fix_scale.bias"] = inp_chkp["head.fix_scale.bias"][REARRANGE_INDICES]

#     return inp_chkp

In [2]:
POSTFIX = ""
EXP_NAME = "eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_EqualBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromXCV2Best_PseudoF2PT05MT01P04I3_AddRareBirdsNoLeak"
TRAIN_PERIOD = 5
print("Possible checkpoints:\n\n{}".format("\n".join(set([os.path.basename(el) for el in glob(f"../logdirs/{EXP_NAME}/*/checkpoints/*.ckpt*") if "train" not in os.path.basename(el)]))))

Possible checkpoints:

epoch=26-step=12879-valid_roc_auc=0.979.ckpt
epoch=34-step=16450-valid_roc_auc=0.987.ckpt
epoch=20-step=9240-valid_roc_auc=0.986.ckpt
epoch=12-step=5200-valid_roc_auc=0.981.ckpt
epoch=24-step=11625-valid_roc_auc=0.985.ckpt
epoch=37-step=17860-valid_roc_auc=0.987.ckpt
epoch=13-step=6510-valid_roc_auc=0.985.ckpt
epoch=15-step=7440-valid_roc_auc=0.986.ckpt
epoch=25-step=11440-valid_roc_auc=0.986.ckpt
epoch=22-step=10120-valid_roc_auc=0.986.ckpt
epoch=9-step=4000-valid_roc_auc=0.981.ckpt
epoch=27-step=13356-valid_roc_auc=0.979.ckpt
epoch=29-step=14100-valid_roc_auc=0.987.ckpt
epoch=35-step=14400-valid_roc_auc=0.981.ckpt
epoch=19-step=9540-valid_roc_auc=0.980.ckpt
last.ckpt


In [3]:
conf_path = glob(f"../logdirs/{EXP_NAME}/code/*train_configs*.py")
assert len(conf_path) == 1
conf_path = conf_path[0]
!cat {conf_path}

from glob import glob

import torch

from code_base.augmentations.transforms import BackgroundNoise, OneOf
from code_base.callbacks import ROC_AUC_Score
from code_base.datasets import WaveAllFileDataset, WaveDataset
from code_base.forwards import MultilabelClsForwardLongShort
from code_base.models import RandomFiltering, WaveCNNAttenClasifier
from code_base.train_functions.train_lightning import lightning_training

B_S = 64
TRAIN_PERIOD = 5.0
N_EPOCHS = 50
ROOT_PATH = "data/train_audio"
LATE_NORMALIZE = True
MAXIMIZE_METRIC = True
MAIN_METRIC = "valid_roc_auc"
PATH_TO_JSON_MAPPING = (
    "data/bird2int_2025.json"
)
PRECOMPUTE = False
REPLACE_PATHES = ("train_audio", "train_features")
DEBUG = False
N_CORES = 12

CONFIG = {
    "seed": 1243,
    "df_path": "data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025_hdf5_fixedaudiometa_h5pyDur.csv",
    "split_path": "data/cv_split_base_and_prev_comps_XCsnipet28032025_group_allrarebirds_hdf5_noleak.npy",
    "exp_name": "eca_

In [4]:
CONFIG = {
    # Main
    "run_validation": True,
    "run_test": False,
    # Inference Class
    "use_sigmoid": False,
    "use_compiled_fp16": False,
    "aggregate_preds": True,
    # Data config
    "train_df_path": "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025_hdf5_fixedaudiometa_h5pyDur.csv",
    "split_path": "../data/cv_split_base_and_prev_comps_XCsnipet28032025_group_allrarebirds_hdf5_noleak.npy",
    "n_folds":5,
    "train_data_root":"../data/train_audio",
    "test_data_root":"../data/train_soundscapes/*.ogg",
    "label_map_data_path": "../data/bird2int_2025.json",
    "scored_birds_path":"../data/sb_2025.json", 
    "lookback":None,
    "lookahead":None,
    "segment_len":5,
    "step": None,
    "late_normalize": True,
    "add_dataset_config": {
        "filename_change_mapping": {
            "base": "train_audio",
            "train_audio": "train_audio",
            "add_train_audio_from_prev_comps": "add_train_audio_from_prev_comps",
            "add_train_audio_from_xeno_canto_28032025": "add_train_audio_from_xeno_canto_28032025",
            "csa_train_audio_parsed_11052025": "csa_train_audio_parsed_11052025",
            "add_train_audio_from_xeno_canto_11052025": "add_train_audio_from_xeno_canto_11052025",
        },
        "ignore_setting_dataset_value": True,
        "duration_col": "duration_s_h5py"
    },
    # Model config
    "exp_name":EXP_NAME,
    "model_class": WaveCNNAttenClasifier,
    "model_config": dict(
        backbone="eca_nfnet_l0",
        mel_spec_paramms={
            "sample_rate": 32000,
            "n_mels": 128,
            "f_min": 20,
            "n_fft": 2048,
            "hop_length": 512,
            "normalized": True,
        },
        head_config={
            "p": 0.5,
            # "num_class": 220,
            "num_class": 206,
            "train_period": TRAIN_PERIOD,
            "infer_period": TRAIN_PERIOD,
            "output_type": "clipwise_pred_long",
        },
        exportable=True,
        fixed_amplitude_to_db=True
    ),
    "chkp_name":"last.ckpt",
    "swa_checkpoint_regex": r'(?P<key>\w+)=(?P<value>[\d.]+)(?=\.ckpt|$)',
    "swa_sort_rule": lambda x: -float(x["valid_roc_auc"]),
    "delete_prefix": "model.",
    "n_swa_models": 1,
    "model_output_key": None,
    # "prune_checkpoint_func": prune_checkpoint_rule,

    # Compilation config
    "use_openvino": True,
    "use_fp16": True,
    "folds_to_onnx": [0, 1, 2, 3, 4],
    "final_activation": None,

    # Information config
    "exp_info": {
        "exp_name": EXP_NAME,
        "chkp_name": "Best",
        "comment": "5 sec train audio pred",
    }
}

CONFIG["exp_info"]["split_path"] = os.path.basename(CONFIG["split_path"])

if CONFIG.get("use_sed_mode", False):
    assert CONFIG["step"] is not None
else:
    assert CONFIG["step"] is None
    
if "folds" not in CONFIG:
    CONFIG["folds"] = list(range(CONFIG["n_folds"]))
    

In [5]:
CONFIG["exp_info"]

{'exp_name': 'eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_EqualBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromXCV2Best_PseudoF2PT05MT01P04I3_AddRareBirdsNoLeak',
 'chkp_name': 'Best',
 'comment': '5 sec train audio pred',
 'split_path': 'cv_split_base_and_prev_comps_XCsnipet28032025_group_allrarebirds_hdf5_noleak.npy'}

# Data

In [6]:
bird2id = load_json(CONFIG["label_map_data_path"])

In [7]:
if CONFIG["run_validation"]:
    df = pd.read_csv(CONFIG["train_df_path"])
    split = np.load(CONFIG["split_path"], allow_pickle=True)
    val_df = [df.iloc[split[i][1]].reset_index(drop=True) for i in CONFIG["folds"]]

In [8]:
if CONFIG["run_test"]:
    test_au_pathes = glob(CONFIG["test_data_root"])[:20]

    test_df = pd.DataFrame({
        "filename": test_au_pathes,
        "duration_s": [librosa.get_duration(filename=el) for el in test_au_pathes]
    })

In [9]:
if CONFIG["run_validation"]:
    val_ds_conig = {
        "root": CONFIG["train_data_root"],
        "label_str2int_mapping_path": CONFIG["label_map_data_path"],
        "use_audio_cache": True,
        "n_cores": 64,
        "verbose": False,
        "segment_len": CONFIG["segment_len"],
        "lookback":CONFIG["lookback"],
        "lookahead":CONFIG["lookahead"],
        "sample_id": None,
        "late_normalize": CONFIG["late_normalize"],
        "step": CONFIG["step"],
        # "validate_sr": 32_000,
    }
    if CONFIG.get("add_dataset_config") is not None:
        val_ds_conig.update(CONFIG["add_dataset_config"])
if CONFIG["run_test"]:
    ds_config_test = {
       "root": "",
       "label_str2int_mapping_path": CONFIG["label_map_data_path"],
       "n_cores": 64,
       "use_audio_cache": True,
       "test_mode": True,
       "segment_len": CONFIG["segment_len"],
       "lookback":CONFIG["lookback"],
       "lookahead":CONFIG["lookahead"],
        "sample_id": None,
        "late_normalize": CONFIG["late_normalize"],
        "step": CONFIG["step"],
        "validate_sr": 32_000
    }
    if CONFIG.get("add_dataset_config") is not None:
        val_ds_conig.update(CONFIG["add_dataset_config"])
loader_config = {
    "batch_size": 64,
    "drop_last": False,
    "shuffle": False,
    "num_workers": 0,
}

In [10]:
if CONFIG["run_test"]:
    ds_test = WaveAllFileDataset(df=test_df, **ds_config_test)
if CONFIG["run_validation"]:
    ds_val = [WaveAllFileDataset(df=df, **val_ds_conig) for df in val_df]

Validating that all files exist


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6239/6239 [00:00<00:00, 351129.31it/s]


Validating that all files exist


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10429/10429 [00:00<00:00, 448304.31it/s]


Validating that all files exist


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5443/5443 [00:00<00:00, 640704.89it/s]


Validating that all files exist


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5924/5924 [00:00<00:00, 619488.32it/s]


Validating that all files exist


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7837/7837 [00:00<00:00, 356299.43it/s]


In [11]:
if CONFIG["run_validation"]:
    loader_val = [torch.utils.data.DataLoader(
        ds,
        **loader_config,
    )for ds in ds_val]
if CONFIG["run_test"]:
    loader_test = torch.utils.data.DataLoader(
        ds_test,
        **loader_config,
    )

In [12]:
len(loader_val)

5

In [13]:
val_ds_conig

{'root': '../data/train_audio',
 'label_str2int_mapping_path': '../data/bird2int_2025.json',
 'use_audio_cache': True,
 'n_cores': 64,
 'verbose': False,
 'segment_len': 5,
 'lookback': None,
 'lookahead': None,
 'sample_id': None,
 'late_normalize': True,
 'step': None,
 'filename_change_mapping': {'base': 'train_audio',
  'train_audio': 'train_audio',
  'add_train_audio_from_prev_comps': 'add_train_audio_from_prev_comps',
  'add_train_audio_from_xeno_canto_28032025': 'add_train_audio_from_xeno_canto_28032025',
  'csa_train_audio_parsed_11052025': 'csa_train_audio_parsed_11052025',
  'add_train_audio_from_xeno_canto_11052025': 'add_train_audio_from_xeno_canto_11052025'},
 'ignore_setting_dataset_value': True,
 'duration_col': 'duration_s_h5py'}

# Model

In [14]:
def create_model_and_upload_chkp(
    model_class,
    model_config,
    model_device,
    model_chkp_root,
    model_chkp_basename=None,
    model_chkp_regex=None,
    delete_prefix=None,
    swa_sort_rule=None,
    n_swa_to_take=3,
    prune_checkpoint_func=None
):
    if model_chkp_basename is None:
        basenames = os.listdir(model_chkp_root)
        checkpoints = []
        for el in basenames:
            matches = re.findall(model_chkp_regex, el)
            if not matches:
                continue
            parsed_dict = {key: value for key, value in matches}
            parsed_dict["name"] = el
            checkpoints.append(parsed_dict)
        print("SWA checkpoints")
        pprint(checkpoints)
        checkpoints = sorted(checkpoints, key=swa_sort_rule)
        checkpoints = checkpoints[:n_swa_to_take]
        print("SWA sorted checkpoints")
        pprint(checkpoints)
        if len(checkpoints) > 1:
            checkpoints = [
                torch.load(os.path.join(model_chkp_root, el["name"]), map_location="cpu")["state_dict"] for el in checkpoints
            ]
            t_chkp = avarage_weights(
                nn_weights=checkpoints,
                delete_prefix=delete_prefix
            )
        else:
            chkp_path = os.path.join(model_chkp_root, checkpoints[0]["name"])
            print("vanilla model")
            print("Loading", chkp_path)
            t_chkp = torch.load(
                chkp_path, 
                map_location="cpu"
            )["state_dict"]
            if delete_prefix is not None:
                t_chkp = delete_prefix_from_chkp(t_chkp, delete_prefix)
    else:
        chkp_path = os.path.join(model_chkp_root, model_chkp_basename)
        print("vanilla model")
        print("Loading", chkp_path)
        t_chkp = torch.load(
            chkp_path, 
            map_location="cpu"
        )["state_dict"]
        if delete_prefix is not None:
            t_chkp = delete_prefix_from_chkp(t_chkp, delete_prefix)

    if prune_checkpoint_func is not None:
        t_chkp = prune_checkpoint_func(t_chkp)
    t_model = model_class(**model_config, device=model_device) 
    print("Missing keys: ", set(t_model.state_dict().keys()) - set(t_chkp))
    print("Extra keys: ",  set(t_chkp) - set(t_model.state_dict().keys()))
    t_model.load_state_dict(t_chkp, strict=False)
    t_model.eval()
    return t_model

In [15]:

model = [create_model_and_upload_chkp(
    model_class=CONFIG["model_class"],
    model_config=CONFIG['model_config'],
    model_device=get_device(),
    model_chkp_root=f"../logdirs/{CONFIG['exp_name']}/fold_{m_i}/checkpoints",
    # model_chkp_root=f"../logdirs/{CONFIG['exp_name']}/checkpoints",
    model_chkp_basename=CONFIG["chkp_name"] if CONFIG["swa_checkpoint_regex"] is None else None,
    model_chkp_regex=CONFIG.get("swa_checkpoint_regex"),
    swa_sort_rule=CONFIG.get("swa_sort_rule"),
    n_swa_to_take=CONFIG.get("n_swa_models", 3),
    delete_prefix=CONFIG.get("delete_prefix"),
    prune_checkpoint_func=CONFIG.get("prune_checkpoint_func")
) for m_i in range(CONFIG["n_folds"])]


SWA checkpoints
[{'name': 'epoch=24-step=11625-valid_roc_auc=0.985.ckpt',
  'valid_roc_auc': '0.985'},
 {'name': 'epoch=13-step=6510-valid_roc_auc=0.985.ckpt',
  'valid_roc_auc': '0.985'},
 {'name': 'epoch=15-step=7440-valid_roc_auc=0.986.ckpt',
  'valid_roc_auc': '0.986'}]
SWA sorted checkpoints
[{'name': 'epoch=15-step=7440-valid_roc_auc=0.986.ckpt',
  'valid_roc_auc': '0.986'}]
vanilla model
Loading ../logdirs/eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_EqualBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromXCV2Best_PseudoF2PT05MT01P04I3_AddRareBirdsNoLeak/fold_0/checkpoints/epoch=15-step=7440-valid_roc_auc=0.986.ckpt
STFT kernels created, time used = 0.7907 seconds
Missing keys:  set()
Extra keys:  {'_forward.batch_aug.hann_window'}
SWA checkpoints
[{'name': 'epoch=9-step=4000-valid_roc_auc=0.981.ckpt',
  'valid_roc_auc': '0.981'},
 {'name': 'epoch=35-step=14400-valid_roc_auc=0.981.ckpt',
  'valid_roc_auc': '0.981'},
 {'name': 'epoch=12-step=5200-valid_roc_auc=0.981

In [16]:
# !rm ../logdirs/convnextv2_tiny_fcmae_ft_in22k_in1k_384_Exp_noamp_64bs_5sec_mixupP05_balancedSampler_lr1e4_CosineLREpoch50_ValV2_202xXcAddDataNoAddSecLabels_BackGroundSoundScapeP05_FocalLoss_FromPretrainFinalV1/onnx_ensem_3first_folds -rf

In [17]:
# os.environ["OMP_NUM_THREADS"]="10"

if CONFIG.get("folds_to_onnx") is not None:
    if isinstance(CONFIG["folds_to_onnx"], list):
        print("Mutli Fold Model")
        exportable_ensem = ONNXEnsemble(
            model_class=CONFIG["model_class"],
            configs=[deepcopy(CONFIG['model_config']) for _ in CONFIG["folds_to_onnx"]],
            final_activation=CONFIG.get("final_activation", None),
            extract_spec_ones=True
        )
        assert len(exportable_ensem.models) == len(model)
        for model_id in range(len(exportable_ensem.models)):
            exportable_ensem.models[model_id].load_state_dict(model[model_id].state_dict())
            exportable_ensem.models[model_id].eval()
        exportable_ensem.eval()
        convert_to_onnx(
            model_to_convert=exportable_ensem,
            sample_input=torch.randn(5, TRAIN_PERIOD * 32_000),
            base_path=f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_5first_folds" + POSTFIX,
            use_fp16=CONFIG.get("use_fp16", False),
            use_openvino=CONFIG.get("use_openvino", False),
            opset_version=12
        )

    elif isinstance(CONFIG["folds_to_onnx"], int):
        print("Solo Fold Model")
        for fold_id in CONFIG["folds_to_onnx"]:
            exportable_ensem = ONNXEnsemble(
                model_class=CONFIG["model_class"],
                configs=[deepcopy(CONFIG['model_config'])],
                final_activation=CONFIG.get("final_activation", None)
            )
            # for model_id in range(len(exportable_ensem.models)):
            exportable_ensem.models[0].load_state_dict(model[fold_id].state_dict())
            
            exportable_ensem.eval()
            convert_to_onnx(
                model_to_convert=exportable_ensem,
                sample_input=torch.randn(5, TRAIN_PERIOD * 32_000),
                base_path=f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_fold{fold_id}",
                use_fp16=CONFIG.get("use_fp16", False),
                use_openvino=CONFIG.get("use_openvino", False),
                opset_version=12
            )

    else:
        raise ValueError(f"{type(CONFIG['folds_to_onnx'])} - unsupported type")

Mutli Fold Model
STFT kernels created, time used = 0.0349 seconds
STFT kernels created, time used = 0.0375 seconds
STFT kernels created, time used = 0.0386 seconds
STFT kernels created, time used = 0.0384 seconds
STFT kernels created, time used = 0.0382 seconds


/home/vsydorskyi/src/bird_clef_2025/.venv/lib/python3.11/site-packages/nnAudio/features/stft.py:283: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.num_samples < self.pad_amount:
/home/vsydorskyi/src/bird_clef_2025/.venv/lib/python3.11/site-packages/torch/onnx/_internal/jit_utils.py:307: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/home/vsydorskyi/src/bird_clef_2025/.venv/lib/python3.11/site-packages/torch/onnx/symbolic_helper.py:1515: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'batch_norm' is

Converting ONNX to OpenVINO


<frozen importlib.util>:262: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression by removing argument "compress_to_fp16" or set it to false "compress_to_fp16=False".
Find more information about compression to FP16 at https://docs.openvino.ai/2023.0/openvino_docs_MO_DG_FP16_Compression.html
[ SUCCESS ] XML file: ../logdirs/eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_EqualBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromXCV2Best_PseudoF2PT05MT01P04I3_AddRareBirdsNoLeak/onnx_ensem_5first_folds_openvino_fp16/model_simpl.xml
[ SUCCESS ] BIN file: ../logdirs/eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_EqualBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromXCV2Best_PseudoF2PT05MT01P04I3_AddRareBirdsNoLeak/onnx_ensem_5first_folds_openvino_fp16/model_simpl.bin


# Inference Class

In [18]:
inference_class = BirdsInference(
    device="cuda",
    verbose_tqdm=True,
    use_sigmoid=CONFIG["use_sigmoid"],
    model_output_key=CONFIG["model_output_key"],
    aggregate_preds=CONFIG.get("aggregate_preds", True)
)

# Val Predict

In [19]:
if CONFIG["run_validation"]:
    val_tgts, val_preds = inference_class.predict_val_loaders(
        nn_models=model,
        data_loaders=loader_val
    )
    if CONFIG.get("scored_birds_path", None):
        print("Extracting scored_bird_ids indices")
        scored_bird = load_json(CONFIG["scored_birds_path"])
        scored_bird_ids = [bird2id[el] for el in scored_bird]
        val_tgts = [el[:,scored_bird_ids] for el in val_tgts]
        val_preds = [el[:,scored_bird_ids] for el in val_preds]
    cmaps = [
        score_numpy(gt, pr) for gt, pr in zip(val_tgts, val_preds)
    ]

    print(
        f"Folds Roc Auc: {cmaps}\n"
        f"Mean Roc Auc: {np.mean(cmaps)}"
    )

  0%|                                                                                                                                                                                      | 0/684 [00:00<?, ?it/s]/home/vsydorskyi/src/bird_clef_2025/.venv/lib/python3.11/site-packages/audioread/rawread.py:16: DeprecationWarning: 'aifc' is deprecated and slated for removal in Python 3.13
  import aifc
/home/vsydorskyi/src/bird_clef_2025/.venv/lib/python3.11/site-packages/audioread/rawread.py:17: DeprecationWarning: 'audioop' is deprecated and slated for removal in Python 3.13
  import audioop
/home/vsydorskyi/src/bird_clef_2025/.venv/lib/python3.11/site-packages/audioread/rawread.py:19: DeprecationWarning: 'sunau' is deprecated and slated for removal in Python 3.13
  import sunau
 83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                             | 568/684 [14:04<03:28,  1.80s/it]Not

Extracting scored_bird_ids indices
Folds Roc Auc: [0.9861931152398505, 0.98126765258526, 0.9801543369552962, 0.9866356732929329, 0.9861170584425252]
Mean Roc Auc: 0.9840735673031731


In [20]:
os.makedirs(os.path.join("../data/predictions/", EXP_NAME + POSTFIX))
np.save(
    os.path.join("../data/predictions/", EXP_NAME + POSTFIX, "tgts.npy"),
    np.array(val_tgts, dtype=object)
)
np.save(
    os.path.join("../data/predictions/", EXP_NAME + POSTFIX, "preds.npy"),
    np.array(val_preds, dtype=object)
)

In [ ]:
# core = ov.Core()
# model = core.read_model(model=f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_fold0_openvino_fp16/model_simpl.xml")
# compiled_model = core.compile_model(model=model, device_name="CPU")

In [ ]:
# if CONFIG["run_validation"]:
#     val_tgts, val_preds = inference_class.predict_val_loaders(
#         nn_models=[compiled_model],
#         data_loaders=loader_val,
#         is_openvino_model=True
#     )
#     if CONFIG.get("scored_birds_path", None):
#         print("Extracting scored_bird_ids indices")
#         scored_bird = load_json(CONFIG["scored_birds_path"])
#         scored_bird_ids = [bird2id[el] for el in scored_bird]
#         val_tgts = [el[:,scored_bird_ids] for el in val_tgts]
#         val_preds = [el[:,scored_bird_ids] for el in val_preds]
#     cmaps = [
#         score_numpy(gt, pr) for gt, pr in zip(val_tgts, val_preds)
#     ]

#     print(
#         f"Folds Roc Auc: {cmaps}\n"
#         f"Mean Roc Auc: {np.mean(cmaps)}"
#     )

In [21]:
print(
    EXP_NAME, "\n"
    f"Folds Roc Auc: {cmaps}\n"
    f"Mean Roc Auc: {np.mean(cmaps)}\n"
    f"OOF Roc Auc: {score_numpy(np.concatenate(val_tgts), np.concatenate(val_preds))}"
)

eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_EqualBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromXCV2Best_PseudoF2PT05MT01P04I3_AddRareBirdsNoLeak 
Folds Roc Auc: [0.9861931152398505, 0.98126765258526, 0.9801543369552962, 0.9866356732929329, 0.9861170584425252]
Mean Roc Auc: 0.9840735673031731
OOF Roc Auc: 0.9814578092965329


# Save Results

In [22]:
METRIC_DF_PATH = "../metrics.csv"

In [23]:
# metric_df = pd.DataFrame(columns=(
#     [
#         "exp_name", "chkp_name", "comment"
#     ]
# ))
# metric_df.to_csv(METRIC_DF_PATH, index=False)

In [24]:
metric_df = pd.read_csv(METRIC_DF_PATH)
metric_df

,exp_name,chkp_name,comment,split_path,Mean Roc AUC,OOF Roc AUC,Folds Roc AUC,Public,Public comment
0,convnextv2_tiny_fcmae_ft_in22k_in1k_384_Exp_no...,Best,5 sec train audio pred,cv_split_basev1.npy,0.981120,0.975946,"[0.9781906685043705, 0.9801255959732124, 0.981...",0.820,Openvino FP16 + 5 Folds Mean
1,eca_nfnet_l0_Exp_noamp_64bs_5sec_mixupP05_Rand...,Best,5 sec train audio pred,cv_split_basev1.npy,0.983473,0.977990,"[0.9798682854800943, 0.9841351285853469, 0.986...",0.827,Openvino FP16 + 5 Folds Mean
2,convnextv2_tiny_fcmae_ft_in22k_in1k_384_Exp_no...,Best,5 sec train audio pred,cv_split_base_and_prev_comps.npy,0.980568,0.972509,"[0.9791606125979209, 0.9815606683566805, 0.977...",0.824,Openvino FP16 + 5 Folds Mean
3,convnext_small_fb_in22k_ft_in1k_384_Exp_noamp_...,Best,5 sec train audio pred,cv_split_base_and_prev_comps.npy,0.980107,0.972958,"[0.9778472336379632, 0.9799773879534882, 0.980...",0.826,Openvino FP16 + 5 Folds Mean
4,convnext_small_fb_in22k_ft_in1k_384_Exp_noamp_...,Best,5 sec train audio pred,cv_split_base_and_prev_comps.npy,0.982635,0.979316,"[0.9788666942852188, 0.9827493212685846, 0.983...",0.822,Openvino FP16 + 5 Folds Mean
...,...,...,...,...,...,...,...,...,...
109,eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_Sqrt...,Best,5 sec train audio pred,cv_split_base_and_prev_comps_XCsnipet28032025_...,0.983743,0.981160,"[0.9839831979138076, 0.980150457930731, 0.9811...",NaN,NaN
110,eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_Sqrt...,Best,5 sec train audio pred,cv_split_base_and_prev_comps_XCsnipet28032025_...,0.983792,0.982039,"[0.984326198344153, 0.9824372112488856, 0.9799...",NaN,NaN
111,tf_efficientnetv2_s_in21k_Exp_noamp_128bs_5sec...,Best,5 sec train audio pred,cv_split_base_and_prev_comps_XCsnipet28032025_...,0.984812,0.979793,"[0.9878266431330837, 0.9819271959045125, 0.978...",NaN,NaN
112,eca_nfnet_l0_Exp_noamp_128bs_5sec_BasicAug_Sqr...,Best,5 sec train audio pred,cv_split_base_and_prev_comps_XCsnipet28032025_...,0.983933,0.981359,"[0.9843506495687405, 0.9830198039835275, 0.978...",NaN,NaN


In [25]:
exp_info = deepcopy(CONFIG["exp_info"])

exp_info["Mean Roc AUC"] = np.mean(cmaps)
exp_info["OOF Roc AUC"] = score_numpy(np.concatenate(val_tgts), np.concatenate(val_preds))
exp_info["Folds Roc AUC"] = cmaps

In [26]:
metric_df = pd.concat([metric_df, pd.Series(exp_info).to_frame().T]).reset_index(drop=True)
metric_df

,exp_name,chkp_name,comment,split_path,Mean Roc AUC,OOF Roc AUC,Folds Roc AUC,Public,Public comment
0,convnextv2_tiny_fcmae_ft_in22k_in1k_384_Exp_no...,Best,5 sec train audio pred,cv_split_basev1.npy,0.98112,0.975946,"[0.9781906685043705, 0.9801255959732124, 0.981...",0.820,Openvino FP16 + 5 Folds Mean
1,eca_nfnet_l0_Exp_noamp_64bs_5sec_mixupP05_Rand...,Best,5 sec train audio pred,cv_split_basev1.npy,0.983473,0.97799,"[0.9798682854800943, 0.9841351285853469, 0.986...",0.827,Openvino FP16 + 5 Folds Mean
2,convnextv2_tiny_fcmae_ft_in22k_in1k_384_Exp_no...,Best,5 sec train audio pred,cv_split_base_and_prev_comps.npy,0.980568,0.972509,"[0.9791606125979209, 0.9815606683566805, 0.977...",0.824,Openvino FP16 + 5 Folds Mean
3,convnext_small_fb_in22k_ft_in1k_384_Exp_noamp_...,Best,5 sec train audio pred,cv_split_base_and_prev_comps.npy,0.980107,0.972958,"[0.9778472336379632, 0.9799773879534882, 0.980...",0.826,Openvino FP16 + 5 Folds Mean
4,convnext_small_fb_in22k_ft_in1k_384_Exp_noamp_...,Best,5 sec train audio pred,cv_split_base_and_prev_comps.npy,0.982635,0.979316,"[0.9788666942852188, 0.9827493212685846, 0.983...",0.822,Openvino FP16 + 5 Folds Mean
...,...,...,...,...,...,...,...,...,...
110,eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_Sqrt...,Best,5 sec train audio pred,cv_split_base_and_prev_comps_XCsnipet28032025_...,0.983792,0.982039,"[0.984326198344153, 0.9824372112488856, 0.9799...",NaN,NaN
111,tf_efficientnetv2_s_in21k_Exp_noamp_128bs_5sec...,Best,5 sec train audio pred,cv_split_base_and_prev_comps_XCsnipet28032025_...,0.984812,0.979793,"[0.9878266431330837, 0.9819271959045125, 0.978...",NaN,NaN
112,eca_nfnet_l0_Exp_noamp_128bs_5sec_BasicAug_Sqr...,Best,5 sec train audio pred,cv_split_base_and_prev_comps_XCsnipet28032025_...,0.983933,0.981359,"[0.9843506495687405, 0.9830198039835275, 0.978...",NaN,NaN
113,tf_efficientnetv2_s_in21k_Exp_noamp_64bs_5sec_...,Best,5 sec train audio pred,cv_split_base_and_prev_comps_XCsnipet28032025_...,0.984828,0.979482,"[0.9869987082286011, 0.9840339530739002, 0.978...",NaN,NaN


In [27]:
metric_df.to_csv(METRIC_DF_PATH, index=False)

# Search for Pseudo

In [ ]:
all_val_tgts = np.concatenate(val_tgts)
all_val_preds = np.concatenate(val_preds)

In [ ]:
all_treshes = np.arange(0.05 , 1.0 + 0.05, 0.05)
all_scores = []
for tresh in tqdm(all_treshes):
    all_scores.append(f1_score(
        all_val_tgts,
        all_val_preds > tresh,
        average="macro"
    ))

In [ ]:
best_idx = np.argmax(all_scores)
best_tresh, best_score = all_treshes[best_idx], all_scores[best_idx]

print(f"Best Score = {best_score} and is reached on {best_tresh}")

plt.title("Macro F1 depending on tresh")
plt.plot(all_treshes, all_scores)
plt.show()

# Test Pred

In [ ]:
if CONFIG["run_test"]:
    test_preds, test_preds_long, test_dfidx, test_end = inference_class.predict_test_loader(
        nn_models=model,
        data_loader=loader_test
    )
    test_pred_df = compose_submission_dataframe(
        probs=test_preds,
        dfidxs=test_dfidx,
        end_seconds=test_end,
        filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
        bird2id=bird2id
    )

In [ ]:
# test_pred_df

# ONNX Test Pred

In [ ]:
inference_class_onnx = BirdsInference(
    device="cpu",
    verbose_tqdm=True,
    use_sigmoid=CONFIG["use_sigmoid"],
    model_output_key=CONFIG["model_output_key"],
    use_compiled_fp16=CONFIG["use_compiled_fp16"],
)
# onnx_model = ort.InferenceSession(
#     # f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_2first_folds/model_simpl.onnx"
#     "../logdirs/convnext_small_fb_in22k_ft_in1k_384_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_balancedSampler_Radamlr1e4_CosBatchLR1e6_Epoch50_SpecAugV1_FocalLoss_Full/onnx_ensem/model_simpl.onnx"
# )
# core = ov.Core()
# model = core.read_model(model=f"../logdirs/convnext_small_fb_in22k_ft_in1k_384_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_balancedSampler_Radamlr1e4_CosBatchLR1e6_Epoch50_SpecAugV1_FocalLoss_Full/openvino_ensem/model_simpl.xml")
# compiled_model = core.compile_model(model=model, device_name="CPU")

In [ ]:
test_preds_onnx, test_dfidx_onnx, test_end_onnx = inference_class_onnx.predict_test_loader(
    nn_models=compiled_model,
    data_loader=loader_test,
    is_openvino_model=True,
)

In [ ]:
test_pred_df_onnx = compose_submission_dataframe(
    probs=test_preds_onnx,
    dfidxs=test_dfidx_onnx,
    end_seconds=test_end_onnx,
    filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
    bird2id=bird2id
)

In [ ]:
test_pred_df_onnx.iloc[:,1:].values

In [ ]:
test_pred_df_onnx_fp16 = pd.read_csv("temp_fp16.csv")

In [ ]:
test_pred_df_onnx_fp16.iloc[:,1:].values

In [ ]:
np.abs(test_pred_df_onnx_fp16.iloc[:,1:].values - test_pred_df_onnx.iloc[:,1:].values).max()

In [ ]:
# test_pred_df_onnx.to_csv("temp_fp16.csv", index=False)

In [ ]:
plt.hist(test_pred_df_onnx.iloc[:,1:].values.flatten())

In [ ]:
if CONFIG["run_test"]:
    inference_class_onnx = BirdsInference(
        device="cpu",
        verbose_tqdm=True,
        use_sigmoid=CONFIG["use_sigmoid"],
        model_output_key=CONFIG["model_output_key"],
        
    )
    onnx_model = ort.InferenceSession(
        f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_2first_folds/model_simpl.onnx"
    )
    test_preds_onnx, test_preds_long_onnx, test_dfidx_onnx, test_end_onnx = inference_class_onnx.predict_test_loader(
        nn_models=onnx_model,
        data_loader=loader_test,
        is_onnx_model=True
    )
    test_pred_df_onnx = compose_submission_dataframe(
        probs=test_preds_onnx,
        dfidxs=test_dfidx_onnx,
        end_seconds=test_end_onnx,
        filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
        bird2id=bird2id
    )

In [ ]:
test_pred_df_onnx

# Map Predictions

In [ ]:
if CONFIG.get("check_inf_class", False):
    val_ds_check = WaveAllFileDataset(df=val_df[0], **{
           "root": CONFIG["train_data_root"],
           "label_str2int_mapping_path": CONFIG["label_map_data_path"],
           "n_cores": 64,
           "use_audio_cache": True,
           "test_mode": True,
           "segment_len": CONFIG["segment_len"],
           "lookback":CONFIG["lookback"],
           "lookahead":CONFIG["lookahead"],
            "sample_id": None,
            "late_normalize": CONFIG["late_normalize"],
            "step": CONFIG["step"],
        }
    )
    val_loader_check = torch.utils.data.DataLoader(
        val_ds_check,
        **loader_config
    )
    
    test_preds_check, test_preds_long_check, test_dfidx_check, test_end_check = inference_class.predict_test_loader(
        nn_models=model,
        data_loader=val_loader_check
    )
    test_preds_check_grouped = groupby_np_array(
        groupby_f=test_dfidx_check,
        array_to_group=test_preds_check,
        apply_f=stack_and_max_by_samples,
    )
    print(np.allclose(
        test_preds_check_grouped,
        val_preds
    ))